In [27]:
from pytube import YouTube
import os
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *
import requests

import pickle
import sys
from pathlib import Path
from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Conv1D, Dense, concatenate
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox

In [2]:
SAVE_PATH = "C:\\Users\\leena\\Desktop\\Python Projects\\emotion-from-tweet\\notebooks"
filename = "video1.mp4"
link = "https://www.youtube.com/watch?v=6ZfuNTqbHE8&t"
category = 'entertainment'
country = 'in'

In [3]:
subs = "There was an idea, to bring together a group of remarkable people, to see if we could become something more. So when they needed us we could fight the battles that they never could. In time, you'll know what it's like to lose. To feel so desperately that you're right, that you fail all the same. Dread it, Run from it. Destiny still arrives. Evacuate the city. Engage all defences and get this man a shield. Fun isn't something one considers the balance in the universe. But this does put a smile on my face. Who the hell are you guys?"
dialogs = subs.split(". ")
start=[4,20,45,51,61,67,77,80,97,104,135]
end=[18,29,50,56,66,71,79,86,102,111,139]

articles = []
news = []

In [4]:
sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())
tokenizer_path = Path('../datasets/sentiment_analysis/tokenizer.pickle').resolve()
with tokenizer_path.open('rb') as file:
    tokenizer = pickle.load(file)

input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
num_classes = 4
embedding_dim = 500
input_length = 100
lstm_units = 128
lstm_dropout = 0.1
recurrent_dropout = 0.1
spatial_dropout=0.2
filters=64
kernel_size=3

input_layer = Input(shape=(input_length,))
output_layer = Embedding(
  input_dim=input_dim,
  output_dim=embedding_dim,
  input_shape=(input_length,)
)(input_layer)

output_layer = SpatialDropout1D(spatial_dropout)(output_layer)

output_layer = Bidirectional(
LSTM(lstm_units, return_sequences=True,
     dropout=lstm_dropout, recurrent_dropout=recurrent_dropout)
)(output_layer)
output_layer = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                    kernel_initializer='glorot_uniform')(output_layer)

avg_pool = GlobalAveragePooling1D()(output_layer)
max_pool = GlobalMaxPooling1D()(output_layer)
output_layer = concatenate([avg_pool, max_pool])

output_layer = Dense(num_classes, activation='softmax')(output_layer)

model = Model(input_layer, output_layer)

model_weights_path = Path('../models/emotion_recognition/model_weights.h5').resolve()
model.load_weights(model_weights_path.as_posix())
encoder_path = Path('../models/emotion_recognition/encoder.pickle').resolve()
with encoder_path.open('rb') as file:
    encoder = pickle.load(file)

In [32]:
def downloadVideo(link):
    yt = YouTube(link)
    stream = yt.streams.first()
    out_file = stream.download(SAVE_PATH)
    os.rename(out_file, filename)

In [6]:
def splitVideo():
    for i in range(len(dialogs)):
        ffmpeg_extract_subclip(filename, start[i],end[i], targetname=dialogs[i][:8]+".mp4")

In [20]:
def fetchNews(country,category):
    response = requests.get("http://newsapi.org/v2/top-headlines?country="+country+"&category="+category+"&apiKey=469e3b5dc7b44bf6ae90867fe0e8fd5c")
    data = response.json()
    articles = data['articles']
    for article in articles:
        print(article['title'])
        title_parts = article['title'].split(" - ")
        text = title_parts[0] + article['description']
        news.append(text)

In [8]:
def calculateEmotion(selected):
    sequences = [text.split() for text in selected]
    list_tokenized = tokenizer.texts_to_sequences(sequences)
    x_data = pad_sequences(list_tokenized, maxlen=100)

    y_pred = model.predict(x_data)
    emo = dict()
    for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
        print(encoder.classes_[index] + ": " + str(value))
        emo[encoder.classes_[index]]=value

In [9]:
def insert_newlines(string, every=64):
    return '\n'.join(string[i:i+every] for i in range(0, len(string), every))

In [10]:
def generateFinalOutput(videoClip,text, dialog):
    clip = VideoFileClip(videoClip)  
    text = insert_newlines(text)
    dialog = insert_newlines(dialog)
    time = clip.duration
    txt_clip = TextClip(text, fontsize = 22, color = 'white') 
    txt_clip2 = TextClip(dialog, fontsize = 15, color = 'yellow')
    txt_clip = txt_clip.set_pos('top').set_duration(time) 
    txt_clip2 = txt_clip2.set_pos('bottom').set_duration(time) 
    video = CompositeVideoClip([clip, txt_clip,txt_clip2])  
    video.write_videofile('Meme_'+videoClip[:8] + text[:8]+ '.mp4')

In [11]:
def getMeme(dialogs,news):
    movie_line = 0
    pairs = dict()
    global_diff = 6
    for j in range(len(dialogs)):
        selected = dialogs[j]
        sequences = [text.split() for text in selected]
        list_tokenized = tokenizer.texts_to_sequences(sequences)
        x_data = pad_sequences(list_tokenized, maxlen=100)

        y_pred = model.predict(x_data)
        emo = dict()
        for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
            print(encoder.classes_[index] + ": " + str(value))
            emo[encoder.classes_[index]]=value

        min_diff = 3
        news_line = 0
        for i in range(len(news)):
            cleaned_data = news[i]
            sequences = [text.split() for text in cleaned_data]
            list_tokenized = tokenizer.texts_to_sequences(sequences)
            x_data = pad_sequences(list_tokenized, maxlen=100)

            y_pred = model.predict(x_data)

            cur_diff = 0
            for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
                cur_diff += abs(emo[encoder.classes_[index]] - value)

            if cur_diff < min_diff:
                news_line = i
                min_diff = cur_diff

        if min_diff < global_diff:
            movie_line = j
            global_diff = min_diff
            
        disp = articles[news_line]['title'] + " : " + dialogs[j]
        print(articles[news_line]['title'] + " : " + dialogs[j] )
        text = insert_newlines(disp,40)
        generateFinalOutput(dialogs[j][:8] + '.mp4', articles[news_line]['title'], dialogs[j] )
        pairs[j] = news_line
    
    print("Final Output --> "+ news[pairs[movie_line]] + " : " + dialogs[movie_line])

In [12]:
def getDialogMeme(selected):
    sequences = [text.split() for text in selected]
    list_tokenized = tokenizer.texts_to_sequences(sequences)
    x_data = pad_sequences(list_tokenized, maxlen=100)

    y_pred = model.predict(x_data)
    emo = dict()
    for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
        print(encoder.classes_[index] + ": " + str(value))
        emo[encoder.classes_[index]]=value
        
    min_diff = 5
    ans = 0
    for i in range(len(news)):
        cleaned_data = news[i]
        sequences = [text.split() for text in cleaned_data]
        list_tokenized = tokenizer.texts_to_sequences(sequences)
        x_data = pad_sequences(list_tokenized, maxlen=100)

        y_pred = model.predict(x_data)

        cur_diff = 0
        for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
            cur_diff += abs(emo[encoder.classes_[index]] - value)

        if cur_diff < min_diff:
            ans = i
            min_diff = cur_diff
            
    print(news[ans] + " : " + selected)
    generateFinalOutput(selected[:8] + '.mp4', news[ans], selected )

In [26]:
getMeme(dialogs,news)

anger: 0.13230877
fear: 0.75031745
joy: 0.020764206
sadness: 0.0966099


KeyboardInterrupt: 

In [37]:
link = "https://www.youtube.com/watch?v=6ZfuNTqbHE8&t"
textYoutubeLink = widgets.Text(
    value=link,
    placeholder='Enter youtube video link',
    description='Youtube: ',
    disabled=False
)

def clickedDownload(arg): 
    downloadVideo(link)
    
button_download = Button(description = 'Download Video')   
button_download.on_click(clickedDownload)


selCountry = widgets.Select(options=['ae','ar','at','au','be','bg','br','ca','ch','cn','co','cu','cz','de','eg','fr','gb','gr','hk','hu','id','ie','il','in','it','jp','kr','lt','lv','ma','mx','my','ng','nl','no','nz','ph','pl','pt','ro','rs','ru','sa','se','sg','si','sk','th','tr','tw','ua','us','ve','za'],value = 'in', rows = 1, description = 'Country: ')
selCategory = widgets.Select(options=['business','entertainment','general','health','science','sports','technology'],value = 'entertainment', rows = 7, description = 'Category: ')

def clickedNews(arg): 
    fetchNews(selCountry.value,selCategory.value)
    
button_news = Button(description = 'Fetch News')   
button_news.on_click(clickedNews)

dialog_text = "But this does put a smile on my face."
def clickedOneMeme(arg): 
    getDialogMeme(txtml.value)

txtml = widgets.Textarea(value=dialog_text,placeholder = "Type the dialog", description = "Enter dialog: ")
button_meme = Button(description = 'Generate Meme')   
button_meme.on_click(clickedOneMeme)

print("Welcome to Meme generator dashboard!")
display(HBox([textYoutubeLink,button_download]))
display(selCategory)
display(selCountry)
display(button_news)
#display(txtml)
display(HBox([txtml,button_meme]))

Welcome to Meme generator dashboard!


Select(description='Category: ', index=1, options=('business', 'entertainment', 'general', 'health', 'science'…

Select(description='Country: ', index=23, options=('ae', 'ar', 'at', 'au', 'be', 'bg', 'br', 'ca', 'ch', 'cn',…

Button(description='Fetch News', style=ButtonStyle())